MIDA Gondara and Wang(2018) in Python (using PyTorch)
https://arxiv.org/abs/1705.02737
https://gist.github.com/lgondara/18387c5f4d745673e9ca8e23f3d7ebd3 

### Note: Section 1 has been tested, moved to utils.py

# 1. Loading Dataset

## 1.1. Load a dataset and introduce missingness

Dataset used: Shuttle Dataset (https://archive.ics.uci.edu/ml/datasets/Statlog+(Shuttle)

### 1.1.1. Load the dataset and store it as dataframe(numeric)

In [1]:
import pandas as pd
import utils

In [2]:
#Test
filename = "data/shuttle/shuttle_trn"
train_df = utils.get_dataframe_from_csv(filename).iloc[:,:-1]  #remove label

INFO:root:Input filename has to be space separated data


In [3]:
train_df.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,28,0,27,48,22
1,55,0,92,0,0,26,36,92,56
2,53,0,82,0,52,-5,29,30,2
3,37,0,76,0,28,18,40,48,8
4,37,0,79,0,34,-26,43,46,2


### 1.1.2. Inducing missingness

After dataset loading, start with inducing missingness. 

To start off, introduce simple random missing patterns (Missing Completely At Random), i.e. sample half of the variables and set observations in those variables to missing if an appended random uniform vector has value less than a certain threshhold. WIth threshold of 0.2, the procedure should introduce about 20% missingness.

In [4]:
#test
df1 = train_df[:]
df2 = utils.induce_missingness(df1,logger_level=20)

INFO:root: Returning new dataframe with missingness(MCAR) induced
INFO:root: Percentage of NaNs in returned dataframe : 8.79


In [5]:
df1.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,28,0,27,48,22
1,55,0,92,0,0,26,36,92,56
2,53,0,82,0,52,-5,29,30,2
3,37,0,76,0,28,18,40,48,8
4,37,0,79,0,34,-26,43,46,2


In [6]:
df2.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,NaN,0,NaN,NaN,NaN
1,55,0,92,0,0.0,26,36.0,92.0,56.0
2,53,0,82,0,52.0,-5,29.0,30.0,2.0
3,37,0,76,0,28.0,18,40.0,48.0,8.0
4,37,0,79,0,34.0,-26,43.0,46.0,2.0


### 1.1.3. Create Train-Test split

Create 70% training data and 30%  test data which includes missingness and a test data without missingness so we can calculate performance. 

In [7]:
#Test
a,b,c = utils.create_train_test_split(df1)
print(a.head())
print(b.head())
print(c.head())

INFO:root: Returning new dataframe with missingness(MCAR) induced
INFO:root: Percentage of NaNs in returned dataframe : 8.79
INFO:root: Returning train_df, test_df, full_test_df after splitting dataframe in 0.7/0.3 split 
INFO:root: Note: full_test_df is the same as test_df but without NaNs


        0  1    2  3     4  5     6     7    8
7476   55  0   98  0   NaN -4   NaN   NaN  NaN
31355  50 -5  102  2  50.0  0  52.0  53.0  0.0
38462  37  0   77  0  36.0 -2  40.0  41.0  2.0
20525  55 -2   95  0  46.0 -3  40.0  49.0  8.0
34457  55  0   92  8   NaN  0   NaN   NaN  NaN
        0  1   2  3     4   5     6     7     8
15528  45 -1  76  0   NaN -16   NaN   NaN   NaN
14327  37  0  95  0  10.0   7  58.0  84.0  26.0
12125  37  0  75 -4  30.0   0  38.0  44.0   6.0
39952  55  0  96  0  50.0   4  41.0  47.0   6.0
1339   41 -1  76  0   NaN -14   NaN   NaN   NaN
        0  1   2  3   4   5   6   7   8
15528  45 -1  76  0  44 -16  31  32   2
14327  37  0  95  0  10   7  58  84  26
12125  37  0  75 -4  30   0  38  44   6
39952  55  0  96  0  50   4  41  47   6
1339   41 -1  76  0  38 -14  35  37   2


# 2. Modelling

Proceed to modelling.

In R:
Start with initializing 'h2o' package and then reading the training and test datasets as the 'h2o's supported format.
Then run imputation model multiple times as each new start would initialize the weights with different values.<br>
Info at: <br>
[h2o](https://cran.r-project.org/web/packages/h2o/h2o.pdf) package offers an easy to use function for implementing autoencoders. 
More information is available at this [link](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/booklets/DeepLearningBooklet.pdf).

In Python:

In [8]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.functional as F

In [9]:
#Settings for device, randomization seed, default tensor type, kwargs for memory #DevSeedTensKwargs
RANDOM_SEED = 18
np.random.seed(RANDOM_SEED)

if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    kwargs = {'num_workers':4, 'pin_memory' :True}
else:
    device = 'cpu'
    torch.manual_seed(RANDOM_SEED)
    torch.set_default_tensor_type(torch.FloatTensor)
    kwards = {}

In [10]:
import dataset_module

In [26]:
df2.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,NaN,0,NaN,NaN,NaN
1,55,0,92,0,0.0,26,36.0,92.0,56.0
2,53,0,82,0,52.0,-5,29.0,30.0,2.0
3,37,0,76,0,28.0,18,40.0,48.0,8.0
4,37,0,79,0,34.0,-26,43.0,46.0,2.0


In [14]:
trainset = dataset_module.DataSetForImputation(df2)

In [15]:
trainset

Dataframe Size:43500, Perc of NaNs: 8.79

In [19]:
trainset[0]

(tensor([50.0000, 21.0000, 77.0000,  0.0000, 34.5129,  0.0000, 37.1039, 50.9072,
         13.9429]),
 tensor([50.0000, 21.0000, 77.0000,  0.0000, 34.5129,  0.0000, 37.1039, 50.9072,
         13.9429]))

In [144]:
import Modelling
net = Modelling.DenoisingAutoEncoder(len(trainset.variables()))

In [145]:
net

DenoisingAutoEncoder(
  (drop_layer): Dropout(p=0.5)
  (linear_layer_list): ModuleList(
    (0): Linear(in_features=9, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=23, bias=True)
    (2): Linear(in_features=23, out_features=30, bias=True)
    (3): Linear(in_features=30, out_features=23, bias=True)
    (4): Linear(in_features=23, out_features=16, bias=True)
    (5): Linear(in_features=16, out_features=9, bias=True)
  )
)

# 3. Training

In [177]:
import torch.utils.data as td
from torch.optim import Adam

LR = 1e-3
BATCH_SIZE = 16
VARIABLES  = len(trainset.variables()) #9
DATAPOINTS = len(trainset)  #45600

import Modelling
net = Modelling.DenoisingAutoEncoder(len(trainset.variables()))

criterion = nn.MSELoss()
net = net.to(device) 

train_loader = td.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, **kwargs) 
optimizer = Adam(net.parameters(), lr = LR)

LOG_INTERVAL = 2

In [178]:
'''
TO DO:
:- Normalization between 0 and 1 - Error blowing up for some reason
:- (0,1) Normalization for better convergence - how to handle this elegantly while testing, because this has been trained for something between 0 and 1
:- Nesterov Momentum + Adam- Pytorch? Decay factor?
'''
from tqdm import tqdm_notebook as tqdm
def train(start_steps = 0, end_steps = 5, net=None, logger_level = 10):
    import logging
    logger = logging.getLogger()
    logger.setLevel(logger_level)
    
    agg_loss = 0.0
    for epoch in tqdm(range(start_steps, end_steps)):
        net.train()
        #Epoch begins
        for x, d in tqdm(train_loader):
#             x = x/x.sum(0).expand_as(x)  # TO DO : NaN, Explore
#             d = d/d.sum(0).expand_as(d)  # Normalize between 0,1 for better convergence #TO DO
            optimizer.zero_grad()
            x = x.to(device)
            with torch.no_grad():
                d = d.to(device)
            y = net(x)
            loss = torch.sqrt(criterion(y, d)/VARIABLES)   #RMSE Loss   #TO DO /VARIABLES because the value was blowing up-> take this out maybe?
            loss.backward()
            optimizer.step()
            agg_loss += loss.item()
            logging.debug(loss)
            
        if epoch%LOG_INTERVAL == 0:
            print(f"Epoch number:{epoch}  Aggregate loss: {agg_loss/(LOG_INTERVAL*DATAPOINTS):.2f}")  #Determine the correct interval and value for loss
            agg_loss= 0.0
        #Epoch Ends

In [180]:
train(10,40, net, logger_level=20)

Epoch number:10  Aggregate loss: 0.45634250864489323


Epoch number:12  Aggregate loss: 0.8627423513620749


Epoch number:14  Aggregate loss: 0.8285637217253105


Epoch number:16  Aggregate loss: 0.809038672548601


Epoch number:18  Aggregate loss: 0.8000655019612148


...

Epoch number:30  Aggregate loss: 0.7692476523963885


Epoch number:32  Aggregate loss: 0.7665664829522714


Epoch number:34  Aggregate loss: 0.7652386885648487


Epoch number:36  Aggregate loss: 0.7665795712553222


Epoch number:38  Aggregate loss: 0.7658164659944073


In [184]:
train(40,500, net, logger_level=20)

Epoch number:40  Aggregate loss: 0.38083006401719716


Epoch number:42  Aggregate loss: 0.758743081986219


Epoch number:44  Aggregate loss: 0.757156429337359


Epoch number:46  Aggregate loss: 0.7573874940954406


Epoch number:48  Aggregate loss: 0.7547284749485981


...

Epoch number:490  Aggregate loss: 0.7318681191192277


Epoch number:492  Aggregate loss: 0.7352475888372838


Epoch number:494  Aggregate loss: 0.7351116572769208


Epoch number:496  Aggregate loss: 0.7293754829319045


Epoch number:498  Aggregate loss: 0.7346177885121312


Ran for 500 iterations
Observation - quickly converges and plateaus at local minima, maybe need a smaller learning rate after a while? or Nesterov?

In [ ]:
#model saving, re-loading

In [185]:
net = net.eval()

print(net(trainset[0][0]))
print(trainset[0][0])

tensor([ 4.5032e+01, -1.2639e-01,  8.5354e+01,  4.3548e-02,  3.9030e+01,
         6.0109e-01,  4.0049e+01,  4.6558e+01,  6.6741e+00],
       grad_fn=<AddBackward0>)
tensor([50.0000, 21.0000, 77.0000,  0.0000, 34.5129,  0.0000, 37.1039, 50.9072,
        13.9429])


In [186]:
print(net(trainset[5][0]))
print(trainset[5][0])

tensor([ 8.9562e+01,  9.5964e-03,  8.5801e+01,  2.3462e-01,  1.6076e+01,
         1.2874e+00, -1.2152e+00,  6.9935e+01,  7.0976e+01],
       grad_fn=<AddBackward0>)
tensor([85.,  0., 88., -4.,  6.,  1.,  3., 83., 80.])


In [189]:
df2

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,NaN,0,NaN,NaN,NaN
1,55,0,92,0,0.0,26,36.0,92.0,56.0
2,53,0,82,0,52.0,-5,29.0,30.0,2.0
3,37,0,76,0,28.0,18,40.0,48.0,8.0
4,37,0,79,0,34.0,-26,43.0,46.0,2.0
5,85,0,88,-4,6.0,1,3.0,83.0,80.0
6,56,0,81,0,-4.0,11,25.0,86.0,62.0
7,55,-1,95,-3,54.0,-4,40.0,41.0,2.0
8,53,8,77,0,28.0,0,23.0,48.0,24.0
9,37,0,101,-7,28.0,0,64.0,73.0,8.0


In [ ]:
train(500,510, net,  logger_level=20)

In [190]:
# Adjust learning rate when needed
# Running time is slow
# Nesterv and time decay and all that
# Normalization between 0 and 1
# Training time should be less than an hour definitely!